In [1]:
import pandas as pd

DATA = '/kaggle/input/fide-titled-chess-players/titled_players_otb.csv'
df = pd.read_csv(filepath_or_buffer=DATA)
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,standart_rating,rapid_rating,...,WIM,WIM_year,FM,FM_year,CM,CM_year,WGM,WGM_year,WH,WH_year
0,1701991,"Aaberg, Anton",SWE,M,IM,NaN,NaN,NaN,2322,2331,...,False,0,False,0,False,0,False,0,False,0
1,1407589,"Aabling-Thomsen, Jakob",DEN,M,IM,NaN,NaN,NaN,2327,0,...,False,0,False,2009,False,0,False,0,False,0
2,25678191,Aaditya Dhingra,IND,M,IM,NaN,NaN,NaN,2421,2208,...,False,0,False,0,False,2020,False,0,False,0
3,25778293,Aadya Gupta,IND,F,WFM,WFM,NaN,NaN,1968,1748,...,False,0,False,0,False,0,False,0,False,0
4,25991426,Aadya Ranganath,IND,F,WCM,WCM,NaN,NaN,1900,1704,...,False,0,False,0,False,0,False,0,False,0
